# Parte 1: Web Scraping con BeautifulSoup

Utilizando la biblioteca BeautifulSoup en Python, extrae información de la siguiente. Debes extraer la siguiente información:

- El nombre de las rutas que aparecen en la página web.
- Donde esta ubicada la ruta.
- El tipo de ruta. Si esta información añadir "Desconocido".
- Duración de la ruta.
- Los kilometros de la ruta.
- La dificultad de la ruta.
- El esfuerzo de la ruta.
- La descripción de la ruta.
- Tendrás que obtener un DataFrame similar al que observas a continuación

In [ ]:
# antes de empezar importamos las librerías que vamos a usar. 
# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests

# Importar librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None) # para poder visualizar todas las columnas de los DataFrames

In [ ]:
#primero hago el request de la url
url_montañas='https://turismoactiva.com/rutas-de-montana-sierra-de-gredos/'
resp_montañas=requests.get(url_montañas)
#A continuación hago la sopa para poder ir trasteando en los datos
sopa_montañas=BeautifulSoup(resp_montañas.content,'html.parser')

In [282]:
#Creo una lista vacía donde acabaré metiendo las tuplas con la info de cada ruta
lista_todos_datos=[]
#Creo una iteración para sacar cada uno de los datos de la parte de la web que me interesa,
# valores anteriores de 7 no me interesan y sólo me interesan los impares
for i in range (7,33,2):
    #Creo una lista de datos vacía donde meteré los datos limpios de la sopa
    lista_datos=[]
    #Creo una lista de datos sopa vacía, que será donde meta el texto de los datos que me interesa
    lista_datos_sopa=[]
    #Saco los datos de cada una de las rutas
    datos=sopa_montañas.find_all('div', {'class':'elementor-widget-wrap elementor-element-populated'})[i]
    #Elimino los \xa0
    datos=datos.text.replace('\xa0',' ')
    #Elimino los saltos de línea y creo una lista
    datos=datos.split('\n')
    #Comienzo a iterar en la lista datos, donde se encuentran mis datos de cada ruta y varias posiciones de lista vacía
    for dato in datos:
        #Si la longitud de dato es mayor que 0, es decir, no está vacío se meterá en la lista_datos_sopa, sino pasa.
        #Otra opción habría sido hacer datos.remove('')
        if len(dato)>0:
            lista_datos_sopa.append(dato)
            
        else:
            pass
    #Añadimos toda la lista de datos sopa en la lista de datos, para hacer una lista de listas
    lista_datos.append(lista_datos_sopa)
    #Creo una lista de datos final donde meter los datos 
    lista_datos_final=[]
    #Itero en la lista de datos
    for montaña in lista_datos:
        #Hago un try except para evitar errores
        try:
            #Si el tipo de ruta no esta en la posición 2 de la lista montaña
            if 'Tipo de ruta:' not in montaña[2]:
                #Si hay Km: en la posición 3
                if 'Km:' in montaña[3]:
                    for k in range(0,6):
                        #Itero en la lista montaña
                        info=montaña[k]
                        #Intento partir el texto de mi lista en el ': ' y meto en datos final la posición 1
                        try:
                            lista_datos_final.append(info.split(': ')[1])
                        #si no puedo hacer eso meto la info directamente
                        except:
                            lista_datos_final.append(info)
                    #Busco la descripción montaña en el último lugar
                    descripción=montaña[-1]
                    lista_datos_final.append(descripción)
                    #Inserto en la posición 2, la que sería de tipo de dato, el valor de Desconocido
                    lista_datos_final.insert(2,'Desconocido')
                    #Añado en la lista de todos los datos la tupla de lisa de datos final 
                    lista_todos_datos.append(tuple(lista_datos_final))
                #Si hay Km: en la posición 3
                elif 'km:' in montaña[3]:
                    for k in range(0,6):
                        #Itero en la lista montaña
                        info=montaña[k]
                        #Intento partir el texto de mi lista en el ': ' y meto en datos final la posición 1
                        try:
                            lista_datos_final.append(info.split(': ')[1])
                        #si no puedo hacer eso meto la info directamente
                        except:
                            lista_datos_final.append(info)
                    #Busco la descripción montaña en el último lugar
                    descripción=montaña[-1]
                    lista_datos_final.append(descripción)
                    #Inserto en la posición 2, la que sería de tipo de dato, el valor de Desconocido
                    lista_datos_final.insert(2,'Desconocido')
                    #Añado en la lista de todos los datos la tupla de lisa de datos final 
                    lista_todos_datos.append(tuple(lista_datos_final))
                #Si no hay km ni Km en la posicion 3
                else:
                    for k in range(0,6):
                        #Itero en la lista montaña
                        info=montaña[k]
                        #Intento partir el texto de mi lista en el ': ' y meto en datos final la posición 1
                        try:
                            lista_datos_final.append(info.split(': ')[1])
                        #si no puedo hacer eso meto la info directamente
                        except:
                            lista_datos_final.append(info)
                    #Busco la descripción montaña en el último lugar
                    descripción=montaña[-1]
                    lista_datos_final.append(descripción)
                    #Inserto en la posición 2, la que sería de tipo de dato, el valor de Desconocido
                    lista_datos_final.insert(2,'Desconocido')
                    #km está en la posición 6 así que lo guardo en una variable y elimino 'km.'
                    km=lista_datos_final[6].replace('km.','')
                    #Popeo el índice 6 (los km)
                    lista_datos_final.pop(6)
                    #inserto ahora los km en la posición deseada
                    lista_datos_final.insert(4,km)
                    #Añado en la lista de todos los datos la tupla de lisa de datos final 
                    lista_todos_datos.append(tuple(lista_datos_final))
            else:
            #repito el código pero si sí que hubiera tipo de ruta
                for k in range(0,7):
                    info=montaña[k]
                    try:
                        lista_datos_final.append(info.split(': ')[1])
                        
                    except:
                        lista_datos_final.append(info)
                descripción=montaña[-1]
                lista_datos_final.append(descripción)
                lista_todos_datos.append(tuple(lista_datos_final))
                
                    
        except:
            pass

In [ ]:
#Creo una lista con los títulos de las columnas
lista_columnas=['nombre', 'donde','tipo','duracion','km','dificultad','esfuerzo','descripcion']

In [283]:
#Creo el dataframe
df_montañas=pd.DataFrame(lista_todos_datos)
df_montañas.columns=lista_columnas
df_montañas
#Hay 2 que la parte de km, dificultad y esfuerzo están desordenados

,nombre,donde,tipo,duracion,km,dificultad,esfuerzo,descripcion
0,Pico Morenzon,"Sierra de Gredos, salida desde la Plataforma d...",circular.,4 horas.,10,baja. (Dependerá de la estación del año).,bajo – medio,Una ruta perfecta para iniciarse en el mundo d...
1,Pico de la Mira,"Sierra de Gredos, salida desde la Plataforma d...",Ida y vuelta.,7 horas.,16,Media. (Dependerá de la estación del año).,Medio,"El Pico de la Mira, desde este punto tendremos..."
2,Pico Almanzor,"Sierra de Gredos, salida desde la Plataforma d...",Ida y vuelta.,14 horas.,22,Alta. (Dependerá de la estación del año).,Alto,Esta actividad se puede realizar en un solo dí...
3,Laguna Grande,"Sierra de Gredos, Plataforma de Gredos, sector...",Ida y vuelta.,5 horas.,12,baja. (Dependerá de la estación del año).,Bajo – medio.,"Ruta conocida por todos, La Laguna Grande de G..."
4,Cinco Lagunas,"Sierra de Gredos, salida desde Navalperal del ...",Ida y vuelta.,9 horas.,25,media. (Dependerá de la estación del año).,alto,Cinco Lagunas sigue siendo uno de esos sitios ...
5,Circo de Hoya Moros,"Sierra de Bejar, salida desde la Plataforma de...",Ida y vuelta. (Posibilidad de circular por la ...,7 horas.,12,Media. (Dependerá de la estación del año).,medio.,Una de las rutas más características de la Sie...
6,Canchal Negro,"Sierra de Bejar, Gredos Oeste (Salamanca)",Desconocido,"3,5 horas",8,baja,bajo,"Se trata de una ruta facil, bonita y apta para..."
7,El Calvitero,"Sierra de Bejar, Gredos Oeste (Salamanca)",Desconocido,"4,5 horas",14,baja – media,Medio,"Un clásico de la Sierra de Béjar, subir al Cal..."
8,"Canchal de la Ceja, Torreon, Calvitero","Sierra de Béjar, salida desde la plataforma de...",circular.,8 horas.,16,medio. (Dependerá de la estación del año).,Alto,La actividad comienza en la plataforma de las ...
9,Laguna de Barco,"Sierra de Gredos, aparcamiento carretera Nava ...",Ida y vuelta.,7 horas.,21,baja. (Dependerá de la estación del año).,medio.,"La Laguna de Galín Gómez, también conocida com..."


# Parte 2: Obtención de Datos Climatológicos con la API de AEMET

Utiliza la API de AEMET para obtener información climatológica de la Sierra de Gredos. En concreto deberás usar el endpoint de "predicciones-especificas" la predicción de montaña para la sierra de gredos. Debe realizar las siguientes tareas:

- Incluir la temperatura máxima en la Sierra de Gredos.
- Incluir la temperatura mínima en la Sierra de Gredos.
- Incluir la fecha en la que se recopilaron los datos.
- Incluir la sierra de donde vienen los datos.